In [1]:
import chipwhisperer as cw
import sys
sys.path.append("../imports")
from helpers import *
from scheme import Scheme
from taskers import *

scope = cw.scope()
target = cw.target(scope, cw.targets.SimpleSerial)
scope.default_setup()
scope.adc.samples=24400

In [5]:
scope.dis()
target.dis()
exit()

In [2]:
# masking outside component
# function
def setup():
    print(start_time)
    # load(scope,"masked")
    # load(scope, "../c-files/masked-syndrome-CWLITEARM")
    load(scope, "../c-files/testsyn", env)
    pass
    
def execute(index, op, t):
    ["ctrl", "ctrl_mask", "degen_eq", "degen_diff", "mask_diff", "mask_eq", 
                               "mask_rand", "synd_diff", "puf_diff"]
    # runs before each CW command -> returned (should be bytearray) is sent to CW
    # index - info passed from user
    # op - operation name
    # t - iteration # (seperate for each op)
    
    global rand, current_puf
    rand = [random.randrange(2) for _ in range(64)]
    if op == "set1":
        p_temp = set_1
    elif op == "set2":
        p_temp = set_2
    elif op == "set1_3":
        p_temp = set_1d3
    elif op == "set2_3":
        p_temp = set_2d3
    elif op == "set1_FI":
        p_temp = set_1_FI.sample({"index": 10}, byte=False)
    elif op == "set1_err":
        p_temp = p1e.sample(byte=False)
    elif op == "set2_err":
        p_temp = p2e.sample(byte=False)
    elif op == "set1_3_err":
        p_temp = p13e.sample(byte=False)
    elif op == "set2_3_err":
        p_temp = p23e.sample(byte=False)
    elif op == "set1_FI_err":
        p_temp = p1e.sample({"index": 10}, byte=False)
    else:
        raise "a"
    try:
        posture(target,'k', get_bytes(p_temp), 20)
    except Exception as e:
        print("here",p_temp)
        raise e

    current_puf = p_temp
    
    return get_bytes(p_temp + [0] + s.encode(rand))
    

def ending(fdb, op, t):
    # runs after each CW command -> no return
    # fdb - data from CW
    # op - operation name
    # t - iteration # (seperate for each op)
    if (get_bits(fdb)[:64] != current_puf[-64:]):
        print("WRONG!")
        print(fdb)
        print(xor(get_bits(fdb)[:64],current_puf[-64:]))
        print()
    pass


current_puf = None
s = Scheme() # scheme execution helper

# k = [random.randrange(2) for _ in range(64)]
set_1 = [random.randrange(2) for _ in range(127)]
set_1_FI = PUF(set_1)
word = s.encode([random.randrange(2) for _ in range(64)])
set_2 = xor(set_1, word)

set_1d3 = list(set_1)
for i in random.choices(range(127),k=3):
    set_1d3[i] ^= 1
    
set_2d3 = list(set_2)
for i in random.choices(range(127),k=3):
    set_2d3[i] ^= 1
    
rand = None
current_puf = None
prefix_name = "SYNDROME2"
    
p1e = PUF(set_1, 0.3)
p2e = PUF(set_2, 0.3)
p13e = PUF(set_1d3, 0.3)
p23e = PUF(set_2d3, 0.3)

env = ENV()
# operation settings
env.root = "C:/Programs/traces"
env.execute = execute
env.ending = ending
env.operations = operations = ["set1", "set2", "set1_3", "set2_3", "set1_FI", "set1_err", "set2_err", "set1_3_err", "set2_3_err", "set1_FI_err"] # operations list , "set2", "set3"
env.subfolder = "synd1" # subfolder name
env.TRACES_NUM = 350 # number of traces
scope.adc.samples = 24400 # number of samples per trace
env.simulate = False # doesnt do anything for now. should be False
env.flush = True
env.index = 0
env.read = 16
env.desc = ""
env.title = ""
env.name = prefix_name
env.readme = """masking is done inside the component.
1 - set1 vs set2 HD_1=HD_2
2 - set1 vs set1+3err
3 - set1 vs set2+3err
4 - set1 vs set w FI
5-8. same but with natural PUF error
9 - error vs no error PUF


PUF error model:
0.7 stable 10e-5 - 10e-6
0.3 unstable 10e-3 - 3*10e-2
python:

posture(target,'k', get_bytes(p_temp), 20)
return get_bytes(p_temp + [0] + s.encode(rand))

c code:

copying and masking
----- trig high
syndrome();
pompom();
"""


# run loop
start_time = date_str()
print(setup())
time.sleep(0.1)


file_names = [f"file{i}" for i in range(7)]
for file_name in file_names:
    status = traces(file_name, scope, target, env)
    

# 1
env.title = "Constant Helper Data, mask <= $; \{f_{PUF1}\} vs \{f_{PUF2}\}"
env.name = prefix_name + "_1"
env.desc = "HD_1=HD_2, PUF1=PUF2+cw"
split_file(env, file_names, [0, 1], "group1")

# 2
env.title = "Small Error, mask <= $; \{f_{PUF1}\} vs \{f_{PUF2}\}"
env.name = prefix_name + "_2"
env.desc = "|f_{PUF1} - f_{PUF2}| = 3"
split_file(env, file_names, [0, 2], "group2")

# 3
env.title = "Constant Helper Data + 3err, mask <= $; \{f_{PUF1}\} vs \{f_{PUF2}\}"
env.name = prefix_name + "_3"
env.desc = "PUF2=PUF1+cw+err, |err|=3"
split_file(env, file_names, [0, 3], "group3")

# 4
env.title = "Fault Injection, mask <= $; \{f_{PUF1}\} vs \{f_{PUF1}+err\}"
env.name = prefix_name + "_4"
env.desc = "Same HD memory, different input"
split_file(env, file_names, [0, 4], "group4")

# 5
env.title = "Constant Helper Data, mask <= $; \{f_{PUF1}\} vs \{f_{PUF2}\}"
env.name = prefix_name + "_5"
env.desc = "HD_1=HD_2, PUF1=PUF2+cw, with PUF error simulation"
split_file(env, file_names, [5, 6], "group5")

# 6
env.title = "Small Error, mask <= 0; \{f_{PUF1}\} vs \{f_{PUF2}\}"
env.name = prefix_name + "_6"
env.desc = "|f_{PUF1} - f_{PUF2}| = 3, with PUF error simulation"
split_file(env, file_names, [5, 7], "group6")

# 7
env.title = "Constant Helper Data + 3err, mask <= $; \{f_{PUF1}\} vs \{f_{PUF2}\}"
env.name = prefix_name + "_7"
env.desc = "PUF2=PUF1+cw+err, |err|=3, with PUF error simulation"
split_file(env, file_names, [5, 8], "group7")

# 8
env.title = "Fault Injection, mask <= $; \{f_{PUF1}\} vs \{f_{PUF1}+err\}"
env.name = prefix_name + "_8"
env.desc = "Same HD memory, different input, with PUF error simulation"
split_file(env, file_names, [5, 9], "group8")

# 9
env.title = "Perfect VS Simulation PUF, mask <= $; \{f_{PUF1}\} vs \{f_{PUF1}\}"
env.name = prefix_name + "_9"
env.desc = "Same PUF, one sampled with error, other not"
split_file(env, file_names, [0, 5], "group9")


remove_files(env, file_names)

env.name = prefix_name
generate_file(env, globals(), "start_time", "operations")
print("done", date_str())


2024-09-03 20-52
Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more


FileNotFoundError: [Errno 2] No such file or directory: '../hexes/../c-files/testsyn.hex'

In [6]:
for i in range(100000):
    if p1e.sample(byte=False) != set_1:
        print("123")

123
123
123
123
123
123
123
123
123
